# Lösen eines FEM Problems mit UG4

Like in the [previous example](./tutorial-fem-01.ipynb) we solve  
$$\nabla \cdot[-D \nabla u] = f $$
mit Diffusionskonstante $D$ und Quelle $f$ sowie Dirichlet-Randwerten.


### Initialisierung 

In [1]:
InitUG(2, AlgebraType("CPU", 1));  -- Initialize world dimension dim=2 and default algebra type
ug_load_script("ug_util.lua")           -- Load utility scripts (e.g. from from ugcore/scripts)
ug_load_script("util/refinement_util.lua")

* Initializing: paths... done, bridge... done, plugins... done                 *
/Users/anaegel/Software/ug4-git/plugins/amg/lua/lua-include.lua
/Users/anaegel/Software/ug4-git/plugins/LevelSet/lua/lua-include.lua
/Users/anaegel/Software/ug4-git/plugins/FiniteStrainMechanics/lua/lua-include.lua
/Users/anaegel/Software/ug4-git/plugins/Limex/lua/lua-include.lua
/Users/anaegel/Software/ug4-git/plugins/SuperLU/lua/lua-include.lua
/Users/anaegel/Software/ug4-git/plugins/ConvectionDiffusion/lua/lua-include.lua
/Users/anaegel/Software/ug4-git/plugins/Richards/lua/lua-include.lua
/Users/anaegel/Software/ug4-git/plugins/DFT/lua/lua-include.lua
/Users/anaegel/Software/ug4-git/plugins/RepoTrend/lua/lua-include.lua
Loading Plugin Lua Content from /Users/anaegel/Software/ug4-git/plugins/RepoTrend/lua/lua-include.lua
RepoTrend-Plugin: Extended package.path:./?.lua;/usr/local/share/lua/5.1/?.lua;/usr/local/share/lua/5.1/?/init.lua;/usr/local/lib/lua/5.1/?.lua;/usr/local/lib/lua/5.1/?/init.lua;/Users/

## Problemdefinitionen

Die Problemdefinitionen legen wir nun in verschiedene LUA-Tabellen ab. Im Folgenden wird dann mit diesen Tabellen gearbeitet:

In [22]:
SQUARE_CONFIG =
{
    -- Geometrie
    gridName= "grids/unit_square_quad.ugx", --"grids/unit_square_tri.ugx",
    requiredSubsets = {"Inner", "Boundary"},
    numRefs= 1,
    
    -- Randbedingungen
    dirichletData = 
    {
        subsets = "Boundary", callback = "MyDirichletBndCallback",  
    },
    
    -- Parameter fuer PDE    
    diffusion = 1.0,  -- D=1.0
    source = 0.0,   -- f=0
}

-- Callback fuer Randbedingungen
function MyDirichletBndCallback(x, y, t)
    if (y==1) then 	return true, 0.0 
    elseif (y==0) then  return true, math.sin(math.pi*1*x)
    else return false, 0.0 
    end
end


CONFIG=SQUARE_CONFIG

In [33]:
SECTOR_CONFIG =
{
    gridName= "grids/sectorTest.ugx",
    requiredSubsets = {"Inner", "Circle", "Cut"},
    numRefs= 1,
    
    dirichletData = 
    {
        subsets = "Circle, Cut", callback = "SectorDirichletSol",  
    },
    
    diffusion = 1.0,
    source = 0.0, 
    myref = "SectorDirichletSol"
}

-- callback function boundary values (only the ones matching 'dim' are used)
function SectorDirichletSol(x, y, t, si)
    local r = math.sqrt(x*x+y*y);
    local phi = math.atan2(y,x);
    if (phi<0) then phi = phi + 2*math.pi; end
    val=math.pow(r,(2/3))*math.sin(phi/3.0*2);
    return val
end
CONFIG = SECTOR_CONFIG


## Rechengebiet
- Einlesen aus einer Datei:

In [34]:
dom = Domain()
LoadDomain(dom, CONFIG.gridName)

- Test: Sind alle Subsets vorhanden?

In [35]:
print(util.CheckSubsets(dom, CONFIG.requiredSubsets))

true


- Uniforme Verfeinerung

In [36]:
local refiner = GlobalDomainRefiner(dom)
for i=1,CONFIG.numRefs do
    write(i .. " ")
    refiner:refine()
end

1 

- Alternativ geht es auch ganz bequem in einer Zeile:

In [37]:
dom = util.CreateDomain(CONFIG.gridName, CONFIG.numRefs, CONFIG.requiredSubsets)

Loading Domain grids/sectorTest.ugx ... done.
Performing integrity check on domain ... done.
Refining(1): 1 done.


## Ansatzraum
- Verwende Polynome 1. Grades:

In [38]:
-- Setup for FEM approximation space.
approxSpace = ApproximationSpace(dom)
approxSpace:add_fct("c", "Lagrange", 1)  -- Linear ansatz functions

-- More inits.
approxSpace:init_levels()
approxSpace:init_top_surface()
approxSpace:print_statistic()

| ---------------------------------------------------------------------------- |
|  Number of DoFs (All Procs)                                                  |
|  Algebra: Block 1 (divide by 1 for #Index)                                   |
|                                                                              |
|    GridLevel   |       Domain |     0: Inner |    1: Circle |       2: Cut   |
| ---------------------------------------------------------------------------- |
| (lev,    0)    |           30 |            8 |           19 |            3 |
| (lev,    1)    |           95 |           51 |           37 |            7 |
| (lev,    0, g) |           30 |            8 |           19 |            3 |
| (lev,    1, g) |           95 |           51 |           37 |            7 |
| (surf, top)    |           95 |           51 |           37 |            7 |
| ---------------------------------------------------------------------------- |
| For Legend and Options: print_statis

## Diskretisierung

Erzeuge Objekt für eine **Elementdiskretisierung** für die Konvektions-Diffusionsgleichung.

In [39]:
elemDisc = ConvectionDiffusion("c", "Inner", "fe")
elemDisc:set_diffusion(CONFIG.diffusion)

if (CONFIG.source) then
    elemDisc:set_source(CONFIG.source)
end

Erzeuge Objekt für **Randbedingungen**:

In [40]:
dirichletBND = DirichletBoundary()
dirichletBND:add(CONFIG.dirichletData.callback, "c", CONFIG.dirichletData.subsets)

Füge beides zu einer Gebietsdiskretisierung hinzu:

In [41]:
domainDisc = DomainDiscretization(approxSpace)
domainDisc:add(elemDisc)
domainDisc:add(dirichletBND)

## Konfiguration eines iterativen Lösers

Ein Mehrgitterverfahren hat lediglich lineare Komplexität

In [42]:
-- set up solver (using 'util/solver_util.lua')
local solverDesc = {
    type = "bicgstab",
    precond = {
        type = "gmg",
        approxSpace = approxSpace,
        smoother = "sgs",
        baseSolver = "lu"
    }
}
solver = util.solver.CreateSolver(solverDesc)

## Assembliere und löse LGS

In [43]:
A = AssembledLinearOperator(domainDisc)
u = GridFunction(approxSpace)
b = GridFunction(approxSpace)
u:set(0.0)


domainDisc:assemble_linear(A, b)
domainDisc:adjust_solution(u)

solver:init(A, u)
solver:apply(u, b)


   % %%%%%%%%             BiCGStab              %%%%%%%%%%%
   % %%%%%%%%   (Precond: Geometric MultiGrid)  %%%%%%%%%%%
   %   Iter      Defect         Rate 
   %    0:    2.205497e+00      -------
   %    1:    6.968047e-02    3.159399e-02
   %    2:    1.398937e-03    2.007646e-02
   %    3:    3.095830e-05    2.212987e-02
   %    4:    1.937968e-07    6.259928e-03
   % Relative reduction 1.000000e-06 reached after 4 steps.
   % Average reduction over 4 steps: 1.721710e-02
   % %%%%%  Iteration converged  %%%%%



- Ausgabe als vtk bzw. vec-Datei

In [44]:
local solFileName = "u_solution"
WriteGridFunctionToVTK(u, solFileName)
SaveVectorForConnectionViewer(u, solFileName .. ".vec")

## Fehleranalyse
L2-Norm $$\|u-u_h\|_0 := \sqrt{\int_\Omega (u-u_h)^2 }$$

In [45]:
if (CONFIG.myref) then
    err0=L2Error(CONFIG.myref,  u, "c", 1.0, 4)
    print(err0)
end

0.014762761394796


 H1-Norm $$\|u-u_h\|_1 := \sqrt{\int_\Omega (u-u_h)^2+ (\nabla (u-u_h))^2 }$$


In [46]:
if (CONFIG.myref) then
    uref = u:clone()
    Interpolate(CONFIG.myref, uref, "c")
    err1=H1Error(uref, "c",  u, "c", 1.0, "Inner")
    print(err1)
end

0.034137308925209


In [17]:
--[[ Append to file
local file = io.open("results.txt", "w") -- opens a file in append mode
io.output(file) -- 
io.write(err0.."\t"..err1)
io.close(file)
--]] 

### Weitere Ausgaben

In [18]:
local matFileName = "A_matrix.mat"
print("writing stiffness matrix to " .. matFileName)
SaveMatrixForConnectionViewer(u, A, matFileName)

local rhsFileName = "b_rhs"
print("writing rhs to '" .. rhsFileName .. ".*'")
SaveVectorForConnectionViewer(b, rhsFileName.. ".vec")

os.execute('gnuplot < test.gnuplot ')
print("done")

writing stiffness matrix to A_matrix.mat
writing rhs to 'b_rhs.*'
done


In [19]:
SQUARE_CONFIG2 =
{
    -- Geometrie
    gridName= "grids/laplace_sample_grid_2d.ugx",
    requiredSubsets = {"Inner", "Boundary"},
    numRefs= 3,
    
    -- Randbedingungen
    dirichletData = 
    {
        subsets = "Boundary", callback = "MyDirichletBndCallback",  
    },
    
    diffusion = 1.0,
    source = "MySourceCallback", 
    myref = "MyRefCallback",
}

-- Callback fuer Randbedingungen
function MyDirichletBndCallback(x, y, t)
    if (y==1) then 	return true, 0.0 
    elseif (y==0) then  return true, math.sin(math.pi*1*x)
    else return false, 0.0 
    end
end

-- Callback fuer Randbedingungen
function MyDirichletBndCallback(x, y, t)
     return true, 0.0 
end

-- Callback fuer rechte Seite
function MySourceCallback(x, y, t)
    local mu = 1.0
    local nu = 4.0
    local scale =  (mu*mu + nu*nu)*(math.pi)*(math.pi)
    return scale*math.sin(math.pi*mu*x)* math.sin(math.pi*nu*y)
end


-- Callback fuer Referenz
function MyRefCallback(x, y, t)
    local mu = 1.0
    local nu = 4.0
    return math.sin(math.pi*mu*x)* math.sin(math.pi*nu*y)
end


CONFIG=SQUARE_CONFIG2

In [20]:
SECTOR_CONFIG =
{
    gridName= "grids/sector_ref0.ugx",
    requiredSubsets = {"Inner", "circle", "cut"},
    numRefs= 4,
    
    dirichletData = 
    {
        subsets = "circle, cut", callback = "SectorDirichletSol",  
    },
    
    diffusion = 1.0,
    source = 0.0, 
    myref = "SectorDirichletSol"
}

-- callback function boundary values (only the ones matching 'dim' are used)
function SectorDirichletSol(x, y, t, si)
    local r = math.sqrt(x*x+y*y);
    local phi = math.atan2(y,x);
    if (phi<0) then phi = phi + 2*math.pi; end
    val=math.pow(r,(2/3))*math.sin(phi/3.0*2);
    return val
end
CONFIG = SECTOR_CONFIG

In [21]:
--[[ file = io.open("test.gnuplot", "w")
io.output(file)  -- sets the default output file as test.lua
io.write('plot "results.txt" using 1')  -- appends a word test to the last line of the file

-- closes the open file
io.close(file)

os.execute('gnuplot < test.gnuplot ')
--]]